## Group Project - Investment Case Study
   #### Group Members:
    -  Vinayak Bandhu
    -  Sarathbabu Sankaran
    -  Puneet Agarwal
    -  Rakesh Gorai

### Checkpoint 1 : Data Cleaning

In [1]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Numpy, pandas packages

import numpy as np
import pandas as pd

In [3]:
# Reading data from different data files

companies = pd.read_csv('companies.txt', sep='\t', encoding='ISO-8859-1')
rounds2 = pd.read_csv('rounds2.csv', encoding='ISO-8859-1')
mapping = pd.read_csv('mapping.csv')

In [4]:
#Cleaning Data in Dataframes (removing special characters)

companies['permalink'] = companies.permalink.str.encode('utf-8').str.decode('ascii','ignore').str.lower()
rounds2['company_permalink'] = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii','ignore').str.lower()
companies['name'] = companies.name.str.encode('utf-8').str.decode('ascii', 'ignore').str.lower()

In [5]:
# Unique companies present in rounds2 and companies files:

print(rounds2['company_permalink'].nunique())
print(companies['permalink'].nunique())

66368
66368


In [6]:
# Inspect companies dataframe 

print(companies.shape)
print(companies.info())
print(companies.describe())

(66368, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB
None
                           permalink   name                homepage_url  \
count                          66368  66367                       61310   
unique                         66368  66037                       61191   
top     /organization/solace-systems  spire  http://www.askforoffer.com   
freq                               1      4                           5   

       category_list     status country_code state_code       regio

In [7]:
# Inspect rounds2 dataframe

print(rounds2.shape)
print(rounds2.info())
print(rounds2.describe(include = 'all'))

(114949, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB
None
               company_permalink  \
count                     114949   
unique                     66368   
top     /organization/solarflare   
freq                          19   
mean                         NaN   
std                          NaN   
min                          NaN   
25%                          NaN   
50%                          NaN   
75%                          NaN   
max                          NaN   

                                funding_round_permalink funding_round_type  \
count      

In [8]:
# Check to see which column can be used as unique key in companies dataframe

## Check for non-null
print(companies.isnull().sum())

## Check for Number of rows and columns:
print(companies.shape)

## Check for Number of unique values in each columns:
print(companies.nunique())

permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6958
state_code        8547
region            8030
city              8028
founded_at       15221
dtype: int64
(66368, 10)
permalink        66368
name             66037
homepage_url     61191
category_list    27296
status               4
country_code       137
state_code         311
region            1092
city              5111
founded_at        3978
dtype: int64


Clearly from above analysis, `permalink` column having no duplicates and non-null values qualifies to be a unique key.

In [9]:
# Check whether any company in rounds2 which is not present in companies dataframe

companies.loc[~companies['permalink'].isin(rounds2['company_permalink']),:]

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at


In [10]:
# Merge the companies and rounds2 dataframe. Since the name of common column in both the dataframe is not same,
# the name of company_permalink is renamed to permalink to remove the duplication of column in master_frame

master_frame = pd.merge(rounds2.rename(columns={'company_permalink':'permalink'}), companies, how = 'inner', on = 'permalink')

In [11]:
# No of observations in master_frame

master_frame.shape[0]

114949

### Checkpoint 2: Funding Type Analysis

In [12]:
# Inspect the master_dataframe

print(master_frame.shape)
print(master_frame.describe(include = 'all'))
print(master_frame.info())

(114949, 15)
                       permalink  \
count                     114949   
unique                     66368   
top     /organization/solarflare   
freq                          19   
mean                         NaN   
std                          NaN   
min                          NaN   
25%                          NaN   
50%                          NaN   
75%                          NaN   
max                          NaN   

                                funding_round_permalink funding_round_type  \
count                                            114949             114949   
unique                                           114949                 14   
top     /funding-round/6b5d8a372ccac5366e0b65bddbab9f6f            venture   
freq                                                  1              55494   
mean                                                NaN                NaN   
std                                                 NaN                NaN   
min     

#### Cleaning the master_frame dataset

In [13]:
# Storing the number of rows of master_frame Dataset into a temp variable 
# to check to retained rows percentage after cleaning the data
master_frame_nrows=len(master_frame.index)

# calculating number of null values in columns and their percentage
print(master_frame.isnull().sum())

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

permalink                      0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
name                           1
homepage_url                6134
category_list               3410
status                         0
country_code                8678
state_code                 10946
region                     10167
city                       10164
founded_at                 20521
dtype: int64


permalink                   0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
name                        0.00
homepage_url                5.34
category_list               2.97
status                      0.00
country_code                7.55
state_code                  9.52
region                      8.84
city                        8.84
founded_at                 17.85
dtype: float64

In [14]:
# Drop unnecessary columns with high percentage of null values

master_frame.drop(['funding_round_code','homepage_url','state_code','region','city','founded_at'],axis=1,inplace=True)

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

permalink                   0.00
funding_round_permalink     0.00
funding_round_type          0.00
funded_at                   0.00
raised_amount_usd          17.39
name                        0.00
category_list               2.97
status                      0.00
country_code                7.55
dtype: float64

In [15]:
# Remove rows with total number of null values 3 or more
master_frame=master_frame[(master_frame.isnull().sum(axis=1))<3]

# Remove rows with raised_amount_usd as null
master_frame=master_frame[~master_frame.raised_amount_usd.isnull()]

# Remove rows with null value country codes
master_frame=master_frame[~master_frame.country_code.isnull()]

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

permalink                  0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
name                       0.00
category_list              0.65
status                     0.00
country_code               0.00
dtype: float64

In [16]:
# Check for number and percentage of retained rows

print(master_frame.shape[0])
print(round((master_frame.shape[0]/master_frame_nrows)*100,2))

89108
77.52


In [17]:
# Converting the raised_amount_usd from usd to usd million

master_frame[['raised_amount_usd']]=round(master_frame[['raised_amount_usd']].apply(lambda x:x/1000000),2)

In [18]:
# Average funding amount in Venture,Private equity, angel and seed type investments

group_by_funding_round_type = master_frame.groupby('funding_round_type')
average_fund = pd.DataFrame(group_by_funding_round_type['raised_amount_usd'].mean())

#round(master_frame.loc[master_frame['funding_round_type']=='angel'].mean(),2)
round(average_fund.loc[['venture','angel','seed','private_equity']],3).sort_values(by = 'raised_amount_usd', ascending = False)

,raised_amount_usd
funding_round_type,
private_equity,73.619
venture,11.736
angel,0.968
seed,0.748


Here the `venture` is the second highest investment type with raised_amount_mn_usd and falls under the 5 to 15 M USD window of Spark Funds. Hence this is the most suitable investment type for Spark Funds.

### Checkpoint 3: Country Analysis

In [19]:
# Top nine countries(based on the total investment amount each country has received)

#Create temporary dataframe to contain only venture type investments data
venture_investment_frame = master_frame.loc[master_frame['funding_round_type'] == 'venture']

#Create top9 dataframe by grouping the venture_investment_dataframe
group_by_country=venture_investment_frame.groupby('country_code')
top9 = group_by_country['raised_amount_usd'].sum().sort_values(ascending=False).head(9)
print(top9)

country_code
USA    422510.47
CHN     39835.29
GBR     20245.59
IND     14391.91
CAN      9583.32
FRA      7259.50
ISR      6907.50
DEU      6347.04
JPN      3363.70
Name: raised_amount_usd, dtype: float64


Here the `USA`, `GBR` and `IND` are the Top 3 English speaking countries in venture venture type investment. Hence are the most suitable countries where Sparks funds can invest.

### Checkpoint 4: Sector Analysis 1

In [20]:
# Create sector_frame to contain data from countries 'USA', 'GBR' and 'IND'

sector_frame=venture_investment_frame.loc[(venture_investment_frame['country_code'] == 'USA') | (venture_investment_frame['country_code'] == 'GBR') | (venture_investment_frame['country_code'] == 'IND')]

# Create a column primamry_sector by spliting category_list on '|' and taking the left value
sector_frame['primary_sector'] = (sector_frame['category_list'].str.split('|',  expand = True)).iloc[:,0]
sector_frame

,permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,category_list,status,country_code,primary_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10.00,#fame,Media,operating,IND,Media
10,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20.00,h2o.ai,Analytics,operating,USA,Analytics
11,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1.70,h2o.ai,Analytics,operating,USA,Analytics
12,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8.90,h2o.ai,Analytics,operating,USA,Analytics
22,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,17-03-2015,5.00,1 mainstream,Apps|Cable|Distribution|Software,acquired,USA,Apps
28,/organization/10-minutes-with,/funding-round/0faccbbcc5818dc5326469f13f5a8ac8,venture,09-10-2014,4.00,10 minutes with,Education,operating,GBR,Education
34,/organization/1000memories,/funding-round/502bd0e50c27616995e4bdad24605ef8,venture,16-02-2011,2.52,1000memories,Curated Web,acquired,USA,Curated Web
38,/organization/1000museums-com,/funding-round/13be128d655076a025221d7fddc90d68,venture,14-10-2009,0.27,1000museums.com,Curated Web,operating,USA,Curated Web
39,/organization/1000museums-com,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,15-01-2014,2.91,1000museums.com,Curated Web,operating,USA,Curated Web
41,/organization/1000museums-com,/funding-round/89317984885b44f02e4befb9dc2e588c,venture,01-08-2014,1.50,1000museums.com,Curated Web,operating,USA,Curated Web


In [21]:
# Inspect mapping file for null values

len(mapping.index)
print(mapping.isnull().sum())
print(round(100*(mapping.isnull().sum()/len(mapping.index)), 2))

category_list                              1
Automotive & Sports                        0
Blanks                                     0
Cleantech / Semiconductors                 0
Entertainment                              0
Health                                     0
Manufacturing                              0
News, Search and Messaging                 0
Others                                     0
Social, Finance, Analytics, Advertising    0
dtype: int64
category_list                              0.15
Automotive & Sports                        0.00
Blanks                                     0.00
Cleantech / Semiconductors                 0.00
Entertainment                              0.00
Health                                     0.00
Manufacturing                              0.00
News, Search and Messaging                 0.00
Others                                     0.00
Social, Finance, Analytics, Advertising    0.00
dtype: float64


#### Cleaning the mapping frame

In [22]:
# Cleaning the mapping dataframe. The 'na' in each category is replaced with '0' in the dataframe.
# hence changing the '0' to 'na' again.

mapping['category_list'] = mapping['category_list'].str.replace('0', 'na', regex=False)
mapping

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,0,1,0,0,0,0
6,Ad Targeting,0,0,0,0,0,0,0,0,1
7,Advanced Materials,0,0,0,0,0,1,0,0,0
8,Adventure Travel,1,0,0,0,0,0,0,0,0
9,Advertising,0,0,0,0,0,0,0,0,1


In [23]:
# Normalizing the mapping dataframe so that merge operation can be done appropriately.

mapping = pd.melt(mapping, id_vars = ["category_list"], var_name = "main_sector")
mapping

,category_list,main_sector,value
0,NaN,Automotive & Sports,0
1,3D,Automotive & Sports,0
2,3D Printing,Automotive & Sports,0
3,3D Technology,Automotive & Sports,0
4,Accounting,Automotive & Sports,0
5,Active Lifestyle,Automotive & Sports,0
6,Ad Targeting,Automotive & Sports,0
7,Advanced Materials,Automotive & Sports,0
8,Adventure Travel,Automotive & Sports,1
9,Advertising,Automotive & Sports,0


In [24]:
# Removing the unwanted rows which doesn't provide any mapping information
mapping = mapping[~(mapping['value'] == 0)]

# Dropping the value column, since it is not useful.
mapping.drop(['value'],axis=1,inplace=True)
mapping.head

<bound method NDFrame.head of                  category_list                              main_sector
8             Adventure Travel                      Automotive & Sports
14                   Aerospace                      Automotive & Sports
45                        Auto                      Automotive & Sports
46             Automated Kiosk                      Automotive & Sports
47                  Automotive                      Automotive & Sports
57                    Bicycles                      Automotive & Sports
69            Boating Industry                      Automotive & Sports
87                         CAD                      Automotive & Sports
93                        Cars                      Automotive & Sports
188                     Design                      Automotive & Sports
189                  Designers                      Automotive & Sports
213                     Drones                      Automotive & Sports
226          Electric Vehicles    

In [25]:
# Merging the sector data with main sector mapping

sector_frame = pd.merge(sector_frame,mapping.rename(columns={'category_list':'primary_sector'}), how = 'inner', on = 'primary_sector' )

# Dropping rows with primary_sector value null
sector_frame= sector_frame[~sector_frame.primary_sector.isnull()]
sector_frame

<bound method NDFrame.head of                                             permalink  \
0                                 /organization/-fame   
1                                 /organization/90min   
2                                 /organization/90min   
3                                 /organization/90min   
4                       /organization/all-def-digital   
5                           /organization/audiosocket   
6                           /organization/audiosocket   
7                           /organization/audiosocket   
8                                /organization/babble   
9                                /organization/babble   
10                               /organization/babble   
11                               /organization/bedloo   
12                               /organization/bedloo   
13                               /organization/bedloo   
14                           /organization/chefs-feed   
15                  /organization/cipher-online-media   
1

### Checkpoint 5: Sector Analysis 2

In [26]:
# Data frame D1 for country USA and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D1 = sector_frame.loc[(sector_frame['country_code'] == 'USA') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D1_groupBy = D1.groupby('main_sector')
D1['total_amount_invested'] = D1_groupBy['raised_amount_usd'].transform('sum')
D1['total_number_of_investments'] = D1_groupBy['raised_amount_usd'].transform('count')
D1

,permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,total_amount_invested,total_number_of_investments
4,/organization/all-def-digital,/funding-round/452a2342fe720285c3b92e9bd927d9ba,venture,06-08-2014,5.00,all def digital,Media,operating,USA,Media,Entertainment,5099.21,591
14,/organization/chefs-feed,/funding-round/adca195749ae9ace84684723fbe75e5b,venture,26-02-2015,5.00,chefsfeed,Media|Mobile|Restaurants|Technology,operating,USA,Media,Entertainment,5099.21,591
20,/organization/huffingtonpost,/funding-round/7f05940c4d2dfecb8e50a0e5720e5065,venture,01-08-2006,5.00,the huffington post,Media|News|Publishing,acquired,USA,Media,Entertainment,5099.21,591
21,/organization/huffingtonpost,/funding-round/9241ae16e08df17ebdc064e49e23035a,venture,01-09-2007,5.00,the huffington post,Media|News|Publishing,acquired,USA,Media,Entertainment,5099.21,591
25,/organization/matchmine,/funding-round/41ac526630da57ad6eb9d02431b17657,venture,01-09-2007,10.00,matchmine,Media|News|Reviews and Recommendations,closed,USA,Media,Entertainment,5099.21,591
27,/organization/mediabong,/funding-round/9282890ca87072025dc1807f400acee6,venture,13-05-2015,5.00,mediabong,Media|Semantic Search|Video,operating,USA,Media,Entertainment,5099.21,591
28,/organization/newscorporation,/funding-round/8f6d7c4592e43e91e8688ba342bffcb7,venture,08-01-2010,12.50,news corp,Media|News|Publishing,ipo,USA,Media,Entertainment,5099.21,591
29,/organization/nokeena,/funding-round/9225f2db6b1b74892d5de6a8744b94ea,venture,08-08-2008,9.40,ankeena networks,Media|Software,acquired,USA,Media,Entertainment,5099.21,591
31,/organization/nokeena,/funding-round/fb7a617ed6d1a3203024f0f111417bfb,venture,25-06-2009,6.50,ankeena networks,Media|Software,acquired,USA,Media,Entertainment,5099.21,591
34,/organization/plumtv,/funding-round/e5109c28c1b4899b068cfa552850c424,venture,01-09-2009,5.20,plumtv,Media|Television|Web Hosting,closed,USA,Media,Entertainment,5099.21,591


In [27]:
# Data frame D2 for country GBR and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D2 = sector_frame.loc[(sector_frame['country_code'] == 'GBR') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D2_groupBy = D2.groupby('main_sector')
D2['total_amount_invested'] = D2_groupBy['raised_amount_usd'].transform('sum')
D2['total_number_of_investments'] = D2_groupBy['raised_amount_usd'].transform('count')
D2

,permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,total_amount_invested,total_number_of_investments
1,/organization/90min,/funding-round/21a2cbf6f2fb2a1c2a61e04bf930dfe6,venture,06-10-2015,15.00,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Entertainment,482.79,56
2,/organization/90min,/funding-round/bd626ed022f5c66574b1afe234f3c90d,venture,07-05-2013,5.80,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,Media,Entertainment,482.79,56
91,/organization/aihit,/funding-round/40cb08117155daba3aa7ad8b81b41068,venture,07-10-2011,5.50,aihit,Analytics|Artificial Intelligence|Business Int...,acquired,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
219,/organization/brandwatch,/funding-round/09f7932220728f0083982db2fced0518,venture,15-03-2012,6.00,brandwatch,Analytics|Social Media,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
435,/organization/digital-shadows,/funding-round/ca618cebbc7e8ada0921af25e9955166,venture,11-02-2015,8.00,digital shadows,Analytics|Cyber Security|Financial Services|In...,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
443,/organization/duedil,/funding-round/05a71740dfbef5f6cb3d2689d8a7b573,venture,12-04-2013,5.00,duedil,Analytics|Transaction Processing,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
645,/organization/intent-hq,/funding-round/6034decdb9ab2fe0d9d5e0372054712d,venture,01-05-2014,8.01,intent hq,Analytics|Interest Graph|Personalization|Software,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
797,/organization/microsaic,/funding-round/4030e9b265deaf48c66c293c7a4c66d4,venture,28-05-2013,6.52,microsaic,Analytics|Environmental Innovation|Nanotechnol...,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
1018,/organization/rangespan,/funding-round/e6c43f976510b334836a7f80aff0fcb8,venture,07-12-2012,5.00,rangespan,Analytics|E-Commerce|Supply Chain Management,acquired,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133
1120,/organization/smart-cube,/funding-round/321a58595ccc6086785e8ac2f6838b81,venture,28-09-2011,6.41,smart cube,Analytics,operating,GBR,Analytics,"Social, Finance, Analytics, Advertising",1089.39,133


In [28]:
# Data frame D3 for country IND and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D3 = sector_frame.loc[(sector_frame['country_code'] == 'IND') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D3_groupBy = D3.groupby('main_sector')
D3['total_amount_invested'] = D3.groupby('main_sector')['raised_amount_usd'].transform('sum')
D3['total_number_of_investments'] = D3.groupby('main_sector')['raised_amount_usd'].transform('count')
D3

,permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,total_amount_invested,total_number_of_investments
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10.00,#fame,Media,operating,IND,Media,Entertainment,280.83,33
454,/organization/eka-software-solutions,/funding-round/2bbd8112aa5429fdf92f13fcb4254f71,venture,01-01-2005,6.00,eka software solutions,Analytics|Logistics|Software|Storage|Supply Ch...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
455,/organization/eka-software-solutions,/funding-round/7b4c0d627c52b378a7c2b5c2edb10c26,venture,01-01-2009,10.00,eka software solutions,Analytics|Logistics|Software|Storage|Supply Ch...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
723,/organization/loginext-solutions,/funding-round/ae317438267472aa73cb1ac43e66a525,venture,22-09-2015,10.00,loginext solutions,Analytics|Big Data Analytics|Data Visualizatio...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
741,/organization/manthan-systems,/funding-round/34b5d3d2bbd104d20d1a62a27220739f,venture,01-03-2007,5.70,manthan systems,Analytics|Big Data Analytics|Business Intellig...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
742,/organization/manthan-systems,/funding-round/384bb8d8bc2a9a1df097f6a4c60e12dc,venture,09-12-2009,15.00,manthan systems,Analytics|Big Data Analytics|Business Intellig...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
744,/organization/manthan-systems,/funding-round/7db541211550feab15626b11aa479116,venture,15-02-2012,15.00,manthan systems,Analytics|Big Data Analytics|Business Intellig...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
745,/organization/manthan-systems,/funding-round/a08435c935a0a43cd9069ad9480f8bb6,venture,20-11-2009,15.00,manthan systems,Analytics|Big Data Analytics|Business Intellig...,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
793,/organization/microland,/funding-round/11f00668f3548380aa1524abc34ae913,venture,14-01-2005,7.30,microland,Analytics,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60
794,/organization/microland,/funding-round/e58d1c7d9fa42b56c3a7410d061d7916,venture,09-08-2006,11.00,microland,Analytics,operating,IND,Analytics,"Social, Finance, Analytics, Advertising",550.55,60


In [29]:
#Total number of investments (count) and Total amount of investment (in million USD) in USA

print(D1['raised_amount_usd'].count())
print(round(D1['raised_amount_usd'].sum(),2))

12044
107578.4


In [30]:
#Total number of investments (count) and Total amount of investment (in million USD) in GBR

print(D2['raised_amount_usd'].count())
print(round(D2['raised_amount_usd'].sum(),2))

620
5380.22


In [31]:
#Total number of investments (count) and Total amount of investment (in million USD) in IND

print(D3['raised_amount_usd'].count())
print(round(D3['raised_amount_usd'].sum(),2))

328
2949.55


In [32]:
# Getting top 3 main_sectors by number of investments in USA based on raised_amount_usd 

D1_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

main_sector
Others                                     2958
Social, Finance, Analytics, Advertising    2718
Cleantech / Semiconductors                 2305
Name: raised_amount_usd, dtype: int64

The top 3 main_sector based on the count of investments in USA are : `Others`, `Social, Finance, Analytics, Advertising` and `Cleantech / Semiconductors`

In [33]:
# Getting top 3 main_sectors by number of investments in GBR based on raised_amount_usd 

D2_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

main_sector
Others                                     148
Social, Finance, Analytics, Advertising    133
Cleantech / Semiconductors                 128
Name: raised_amount_usd, dtype: int64

The top 3 main_sector based on the count of investments in GBR are : `Others`, `Social, Finance, Analytics, Advertising` and `Cleantech / Semiconductors`

In [34]:
# Getting top 3 main_sectors by number of investments in IND based on raised_amount_usd

D3_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

main_sector
Others                                     110
Social, Finance, Analytics, Advertising     60
News, Search and Messaging                  52
Name: raised_amount_usd, dtype: int64

The top 3 main_sector based on the count of investments in IND are : `Others`, `Social, Finance, Analytics, Advertising` and `News, Search and Messaging`

In [35]:
#Name of the company in USA for "Others" Sector, which is top in amount_invested

D1_groupby_name=D1[D1.main_sector=='Others'].groupby('name')
D1_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
virtustream          64.30
capella photonics    54.97
pmw technologies     54.89
airtight networks    54.20
decarta              52.10
Name: raised_amount_usd, dtype: float64

Clearly `virtustream` has the max investment for "Others" Sector in USA

In [36]:
#Name of the company in GBR for "Others" Sector, which is top in amount invested

D2_groupby_name=D2[D2.main_sector=='Others'].groupby('name')
D2_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
electric cloud             37.00
sensage                    36.25
enigmatec                  32.50
silverrail technologies    29.00
opencloud                  27.97
Name: raised_amount_usd, dtype: float64

Clearly `electric cloud` has the max investment for "Others" Sector in GBR

In [37]:
#Name of the company in IND for "Others" Sector, which is top in amount invested

D3_groupby_name=D3[D3.main_sector=='Others'].groupby('name')
D3_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
firstcry.com         39.0
myntra               38.0
commonfloor          32.9
pepperfry.com        28.0
itzcash card ltd.    25.0
Name: raised_amount_usd, dtype: float64

Clearly `firstcry.com` has the max investment for "Others" Sector in IND

In [38]:
#Name of the company in USA for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D1_groupby_name1=D1[D1.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D1_groupby_name1['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
sst inc. (formerly shotspotter)    67.93
demandbase                         63.00
intacct                            61.80
netbase solutions                  60.60
lotame                             59.70
Name: raised_amount_usd, dtype: float64

Clearly `sst inc. (formerly shotspotter)` has the max investment in "Social, Finance, Analytics, Advertising" in USA

In [39]:
#Name of the company in GBR for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D2_groupby_name2=D2[D2.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D2_groupby_name2['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
celltick technologies    37.50
mythings                 34.00
zopa                     32.90
visualdna                28.55
marketinvoice            25.55
Name: raised_amount_usd, dtype: float64

Clearly `celltick technologies` has the max investment in "Social, Finance, Analytics, Advertising" in GBR

In [41]:
#Name of the company in IND for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D3_groupby_name3=D3[D3.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D3_groupby_name3['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

name
manthan systems               50.70
komli media                   28.00
shopclues.com                 25.00
intarvo                       21.90
grameen financial services    21.56
Name: raised_amount_usd, dtype: float64

Clearly `manthan systems` has the max investment in "Social, Finance, Analytics, Advertising" in IND

### Checkpoint 6: Plots

In [42]:
# Exporting data to Excel to create a tableau connection to plot graphs

master_frame.to_excel(pd.ExcelWriter('Master_Frame_Data.xlsx'),'master_frame_data')
sector_frame.to_excel(pd.ExcelWriter('Sector_Frame_Data.xlsx'),'sector_frame_data')

In [43]:
# Analyse excel for the exported dataframes and, 
# match it with results below to verify proper export of master_frame and sector_frame

# master_frame validation
print(master_frame.shape)
print(master_frame.info())

# sector_frame_validation
print(sector_frame.shape)
print(sector_frame.info())

(89108, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 89108 entries, 0 to 114947
Data columns (total 9 columns):
permalink                  89108 non-null object
funding_round_permalink    89108 non-null object
funding_round_type         89108 non-null object
funded_at                  89108 non-null object
raised_amount_usd          89108 non-null float64
name                       89107 non-null object
category_list              88529 non-null object
status                     89108 non-null object
country_code               89108 non-null object
dtypes: float64(1), object(8)
memory usage: 6.8+ MB
None
(38638, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38638 entries, 0 to 38852
Data columns (total 11 columns):
permalink                  38638 non-null object
funding_round_permalink    38638 non-null object
funding_round_type         38638 non-null object
funded_at                  38638 non-null object
raised_amount_usd          38638 non-null float64
name          